In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#pip install --upgrade ipykernel

In [3]:
#pip install autocorrect


In [4]:
#pip install textblob


In [5]:
#pip install gensim==3.8.3


In [6]:
#!pip install pyLDAvis


In [7]:
#pip install spacy


In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import regex as re
import nltk
import scipy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from autocorrect import Speller
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from gensim.models import CoherenceModel
import gensim
import gensim.corpora as corpora
import spacy

In [9]:
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [10]:
!type -a pip

pip is /Library/Frameworks/Python.framework/Versions/3.9/bin/pip


In [11]:
import pyLDAvis
import pyLDAvis.gensim_models

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [12]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/janine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
merged = pd.read_csv("merged_new.csv")
merged.Content

0     This paper aims to compute the performances of...
1     Renewable energy project development is highly...
2     This study assesses the factors responsible fo...
3     An empirical analysis of the non-linear impact...
4     Climate change is increasingly recognized worl...
                            ...                        
82    This paper analyses the politics of renewable ...
83    Expansion of renewable energies is a central p...
84    Stakeholders strongly influence project succes...
85    This paper aims to analyse reasons for the abs...
86    Citizens own nearly half the renewable energy ...
Name: Content, Length: 87, dtype: object

In [14]:
#display word count
merged['word_count'] = merged['Content'].apply(lambda x: len(str(x).split(" ")))
merged[['Content','word_count']].head()

,Content,word_count
0,This paper aims to compute the performances of...,777
1,Renewable energy project development is highly...,768
2,This study assesses the factors responsible fo...,817
3,An empirical analysis of the non-linear impact...,27
4,Climate change is increasingly recognized worl...,490


# Data Prep

In [15]:
#lowercase content
merged['Content'] = merged['Content'].apply(lambda x: " ".join(x.lower() for x in x.split()))
merged['Content'].head()

0    this paper aims to compute the performances of...
1    renewable energy project development is highly...
2    this study assesses the factors responsible fo...
3    an empirical analysis of the non-linear impact...
4    climate change is increasingly recognized worl...
Name: Content, dtype: object

In [16]:
#remove punctuations
merged['Content'] = merged['Content'].str.replace('[^\w\s]','')
merged['Content'].head()

<>:2: DeprecationWarning: invalid escape sequence \w
<>:2: DeprecationWarning: invalid escape sequence \w
/var/folders/0j/p1sf5x296z57_lsn25m7m2y80000gn/T/ipykernel_24918/509571761.py:2: DeprecationWarning: invalid escape sequence \w
  merged['Content'] = merged['Content'].str.replace('[^\w\s]','')


0    this paper aims to compute the performances of...
1    renewable energy project development is highly...
2    this study assesses the factors responsible fo...
3    an empirical analysis of the nonlinear impacts...
4    climate change is increasingly recognized worl...
Name: Content, dtype: object

In [17]:
#display number of stopwords

from nltk.corpus import stopwords
stop = stopwords.words('english')

merged['stopwords'] = merged['Content'].apply(lambda x: len([x for x in x.split() if x in stop]))
merged[['Content','stopwords']].head()

,Content,stopwords
0,this paper aims to compute the performances of...,319
1,renewable energy project development is highly...,315
2,this study assesses the factors responsible fo...,277
3,an empirical analysis of the nonlinear impacts...,7
4,climate change is increasingly recognized worl...,179


In [18]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [19]:
#remove stopwords
stop = stopwords.words('english')
merged['Content'] = merged['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
merged['Content'].head()

0    paper aims compute performances smaller versio...
1    renewable energy project development highly co...
2    study assesses factors responsible success fai...
3    empirical analysis nonlinear impacts icttrade ...
4    climate change increasingly recognized worldwi...
Name: Content, dtype: object

# Exploratory analysis 

In [20]:
TextBlob(merged['Content'][0]).ngrams(2)

[WordList(['paper', 'aims']),
 WordList(['aims', 'compute']),
 WordList(['compute', 'performances']),
 WordList(['performances', 'smaller']),
 WordList(['smaller', 'version']),
 WordList(['version', 'solana']),
 WordList(['solana', 'power']),
 WordList(['power', 'plant']),
 WordList(['plant', 'half']),
 WordList(['half', 'solar']),
 WordList(['solar', 'field']),
 WordList(['field', '1']),
 WordList(['1', '2']),
 WordList(['2', 'turbines']),
 WordList(['turbines', 'power']),
 WordList(['power', 'cycle']),
 WordList(['cycle', 'built']),
 WordList(['built', 'amman']),
 WordList(['amman', 'maan']),
 WordList(['maan', 'jordan']),
 WordList(['jordan', 'climate']),
 WordList(['climate', 'conditions']),
 WordList(['conditions', 'amman']),
 WordList(['amman', 'maan']),
 WordList(['maan', 'discussed']),
 WordList(['discussed', 'thoroughly']),
 WordList(['thoroughly', 'paper']),
 WordList(['paper', 'furthermore']),
 WordList(['furthermore', 'preliminary']),
 WordList(['preliminary', 'validation']

In [21]:
TextBlob(merged['Content'][0]).ngrams(3)

[WordList(['paper', 'aims', 'compute']),
 WordList(['aims', 'compute', 'performances']),
 WordList(['compute', 'performances', 'smaller']),
 WordList(['performances', 'smaller', 'version']),
 WordList(['smaller', 'version', 'solana']),
 WordList(['version', 'solana', 'power']),
 WordList(['solana', 'power', 'plant']),
 WordList(['power', 'plant', 'half']),
 WordList(['plant', 'half', 'solar']),
 WordList(['half', 'solar', 'field']),
 WordList(['solar', 'field', '1']),
 WordList(['field', '1', '2']),
 WordList(['1', '2', 'turbines']),
 WordList(['2', 'turbines', 'power']),
 WordList(['turbines', 'power', 'cycle']),
 WordList(['power', 'cycle', 'built']),
 WordList(['cycle', 'built', 'amman']),
 WordList(['built', 'amman', 'maan']),
 WordList(['amman', 'maan', 'jordan']),
 WordList(['maan', 'jordan', 'climate']),
 WordList(['jordan', 'climate', 'conditions']),
 WordList(['climate', 'conditions', 'amman']),
 WordList(['conditions', 'amman', 'maan']),
 WordList(['amman', 'maan', 'discussed

In [22]:
#words with highest frequency
freq = pd.Series(' '.join(merged['Content']).split()).value_counts()[:20]
freq

energy           1174
renewable         684
policy            387
project           294
success           280
projects          256
factors           249
development       223
also              222
study             205
important         198
research          193
community         170
technology        169
support           164
local             152
different         148
stakeholders      136
environmental     135
policies          135
dtype: int64

In [23]:
#add new stop words

stop = stopwords.words('english')
newStopWords = ['also','1','2','3','4','eu','renewables','china','csfs','turkey','et','al','different','jordan','maan','anman','important','results','however','paper','new','implications','research','may','factors','study','success','this''csf','project','finding','result','decision','impact''important','solar','wind','renewable','energy','factors','projects','success','study']
stop.extend(newStopWords)

merged['Content'] = merged['Content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
merged['Content'].head()


0    aims compute performances smaller version sola...
1    development highly complex means guaranteed de...
2    assesses responsible failure access programs b...
3    empirical analysis nonlinear impacts icttrade ...
4    climate change increasingly recognized worldwi...
Name: Content, dtype: object

In [24]:
#words with highest frequency
freq = pd.Series(' '.join(merged['Content']).split()).value_counts()[:20]
freq

policy           387
development      223
community        170
technology       169
support          164
local            152
stakeholders     136
policies         135
environmental    135
analysis         130
financial        124
government       123
role             120
economic         118
technologies     114
sector           113
social           109
one              104
power            103
use              101
dtype: int64

In [25]:
TextBlob(merged['Content'][0]).ngrams(2)

[WordList(['aims', 'compute']),
 WordList(['compute', 'performances']),
 WordList(['performances', 'smaller']),
 WordList(['smaller', 'version']),
 WordList(['version', 'solana']),
 WordList(['solana', 'power']),
 WordList(['power', 'plant']),
 WordList(['plant', 'half']),
 WordList(['half', 'field']),
 WordList(['field', 'turbines']),
 WordList(['turbines', 'power']),
 WordList(['power', 'cycle']),
 WordList(['cycle', 'built']),
 WordList(['built', 'amman']),
 WordList(['amman', 'climate']),
 WordList(['climate', 'conditions']),
 WordList(['conditions', 'amman']),
 WordList(['amman', 'discussed']),
 WordList(['discussed', 'thoroughly']),
 WordList(['thoroughly', 'furthermore']),
 WordList(['furthermore', 'preliminary']),
 WordList(['preliminary', 'validation']),
 WordList(['validation', 'exercise']),
 WordList(['exercise', 'performed']),
 WordList(['performed', 'using']),
 WordList(['using', 'measured']),
 WordList(['measured', 'monthly']),
 WordList(['monthly', 'average']),
 WordList

In [26]:
TextBlob(merged['Content'][0]).ngrams(3)

[WordList(['aims', 'compute', 'performances']),
 WordList(['compute', 'performances', 'smaller']),
 WordList(['performances', 'smaller', 'version']),
 WordList(['smaller', 'version', 'solana']),
 WordList(['version', 'solana', 'power']),
 WordList(['solana', 'power', 'plant']),
 WordList(['power', 'plant', 'half']),
 WordList(['plant', 'half', 'field']),
 WordList(['half', 'field', 'turbines']),
 WordList(['field', 'turbines', 'power']),
 WordList(['turbines', 'power', 'cycle']),
 WordList(['power', 'cycle', 'built']),
 WordList(['cycle', 'built', 'amman']),
 WordList(['built', 'amman', 'climate']),
 WordList(['amman', 'climate', 'conditions']),
 WordList(['climate', 'conditions', 'amman']),
 WordList(['conditions', 'amman', 'discussed']),
 WordList(['amman', 'discussed', 'thoroughly']),
 WordList(['discussed', 'thoroughly', 'furthermore']),
 WordList(['thoroughly', 'furthermore', 'preliminary']),
 WordList(['furthermore', 'preliminary', 'validation']),
 WordList(['preliminary', 'valid

In [27]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data prep

In [28]:
#tokenization
TextBlob(merged['Content'][1]).words

#lemmatization
from textblob import Word
merged['Content'] = merged['Content'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
merged['Content'].head()

0    aim compute performance smaller version solana...
1    development highly complex mean guaranteed dec...
2    ass responsible failure access program banglad...
3    empirical analysis nonlinear impact icttrade o...
4    climate change increasingly recognized worldwi...
Name: Content, dtype: object

# Non-Negative Matrix Factorisation
#source https://stackabuse.com/python-for-nlp-topic-modeling/

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [30]:
tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2, stop_words=stop)
doc_term_matrix = tfidf_vect.fit_transform(merged['Content'].values.astype ('U'))

In [31]:
from sklearn.decomposition import NMF

nmf = NMF(n_components=5, random_state=42)
nmf.fit(doc_term_matrix)

NMF(n_components=5, random_state=42)

In [32]:
import random

for i in range(10):
    random_id = random.randint(0,len(tfidf_vect.get_feature_names()))
    print(tfidf_vect.get_feature_names()[random_id])

many
allowed
discus
ensure
interdisciplinary
actually
pointed
draw
indicates
26


In [33]:
first_topic = nmf.components_[0]
top_topic_words = first_topic.argsort()[-10:]

In [34]:
for i in top_topic_words:
    print(tfidf_vect.get_feature_names()[i])

environmental
mix
state
government
country
development
technology
industry
sector
policy


In [35]:
for i,topic in enumerate(nmf.components_):
    print(f'Top 10 words for topic #{i}:')
    print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['environmental', 'mix', 'state', 'government', 'country', 'development', 'technology', 'industry', 'sector', 'policy']


Top 10 words for topic #1:
['process', 'development', 'technology', 'management', 'program', 'partnership', 'social', 'beneficiary', 'local', 'community']


Top 10 words for topic #2:
['partnership', 'green', 'satisfaction', 'relationship', 'proposed', 'network', 'twomode', 'method', 'retrofit', 'stakeholder']


Top 10 words for topic #3:
['market', 'retail', 'portfolio', 'policy', 'performance', 'financial', 'technology', 'preference', 'investor', 'investment']


Top 10 words for topic #4:
['fit', 'expansion', 'power', 'state', 'price', 'promotion', 'electricity', 'capacity', 'plant', 'system']




In [36]:
topic_values = nmf.transform(doc_term_matrix)
merged['Topic'] = topic_values.argmax(axis=1)
merged.head()

,Title,Content,word_count,stopwords,Topic
0,A 140 MW Solar Thermal Plant in Jordan,aim compute performance smaller version solana...,777,319,4
1,A fuzzy levelised energy cost method for renew...,development highly complex mean guaranteed dec...,768,315,2
2,A qualitative factor analysis of renewable ene...,ass responsible failure access program banglad...,817,277,1
3,An empirical analysis of the non-linear impact...,empirical analysis nonlinear impact icttrade o...,27,7,1
4,Assessing Turkey's Climate Change Commitments:...,climate change increasingly recognized worldwi...,490,179,0
